In [35]:
import user_credentials
credentials = user_credentials.main()

# InPort API (Beta) #
###### https://docs.google.com/document/d/1txAdY6YChEcdOoEWquF3GzT6yBvpHOiuOJIQvWNZHaE/edit?tab=t.0#heading=h.mij7lb2wckgv ######

## Loader API Specs ###

### Authentication ###

In [36]:
import requests, json
inport_url = "https://test-www.fisheries.noaa.gov/inport"
inport_response = requests.get(url=inport_url, auth=(credentials["username"], credentials["password"]))
if inport_response.status_code == 200:
    print("Authentication successful!")
elif inport_response.status_code == 401:
    print("Authentication NOT successful!")
else:
    print(f"Something is wrong!! Status Code: {response.status_code}")
inport_response.close()
del inport_response, inport_url
# RESULT ###--->>> Authentication successful!

Authentication successful!


### Get Session ID ###
#### /inport/api/get-session-id ####
#### (POST) ####

In [37]:
import requests, json, copy
inport_url = "https://test-www.fisheries.noaa.gov/inport"
inport_response = requests.get(url=inport_url, auth=(credentials["username"], credentials["password"]))
if inport_response.status_code == 200:
    print("Authentication successful!")
    _url = "https://test-www.fisheries.noaa.gov/inport/api/get-session-id"
    session_id_response = requests.post(_url, json = credentials)
    session_id_dict = copy.deepcopy(session_id_response.json())
    print(session_id_dict)
    session_id_response.close()
    del _url, session_id_response
elif inport_response.status_code == 401:
    print("Authentication NOT successful!")
else:
    print(f"Something is wrong!! Status Code: {response.status_code}")
inport_response.close()
del inport_response, inport_url
# RESULT ###--->>> Authentication successful!
# {'sessionId': 'b7f9fb6bf34a6be8b357020e'}

Authentication successful!
{'sessionId': '22469789aebb6c1fcaad6669'}


### Endpoints ###

### Load InPort XML ###
#### /inport/api/load-inport-xml ####
#### (POST) ####
#### NOTE: Not ready for this yet ####

In [46]:
import requests, json, copy
inport_url = "https://test-www.fisheries.noaa.gov/inport"
inport_response = requests.get(url=inport_url, auth=(credentials["username"], credentials["password"]))
if inport_response.status_code == 200:
    print("Authentication successful!")

    _url = "https://test-www.fisheries.noaa.gov/inport/api/get-session-id"
    session_id_response = requests.post(_url, json = credentials)
    session_id_dict = copy.deepcopy(session_id_response.json())
    #print(session_id_dict)
    session_id_response.close()
    del _url, session_id_response

    from lxml import etree
    from io import StringIO

    xml = r"C:\Users\john.f.kennedy\Documents\ArcGIS\Projects\ArcGIS-XML-to-InPort-XML\inport-xml.xml"
    parser = etree.XMLParser(encoding='UTF-8', remove_blank_text=True)
    tree = etree.parse(xml, parser=parser)

    session_id_dict["xml"] = etree.tostring(tree, encoding='UTF-8', method='xml', xml_declaration=True).decode()
    #print(session_id_dict)
    _url = "https://test-www.fisheries.noaa.gov/inport/api/load-inport-xml"
    response = requests.get(_url, params = session_id_dict)        
    if response.status_code == 200:
        print(response)
        _dict = copy.deepcopy(response.json())
        response.close()
    
        for key in _dict:
            print(key)
            if isinstance(_dict[key], dict):
                for k in _dict[key]:
                    print(f"\t{k}: {_dict[key][k]}")
            elif isinstance(_dict[key], list):
                for i in range(0, len(_dict[key])):
                    for k in _dict[key][i]:
                        print(f"\t{k}: {_dict[key][i][k]}")
                    print()
  
    elif response.status_code == 400:
        print(session_id_dict)    
        print(response)        
    
    else:
        print(f"Something is wrong!! Status Code: {response.status_code}")

    del response, _url
    
elif inport_response.status_code == 401:
    print("Authentication NOT successful!")
else:
    print(f"Something is wrong!! Status Code: {response.status_code}")
inport_response.close()
del inport_response, inport_url
# RESULT ###--->>> Authentication successful!

Authentication successful!
{'sessionId': '7819a106ac3abe6dad9d5e71', 'xml': '<?xml version=\'1.0\' encoding=\'UTF-8\'?>\n<inport-metadata xmlns:xs="http://www.w3.org/2001/XMLSchema" version="1.11" source="https://test-www.fisheries.noaa.gov"><item-identification><catalog-item-id>75728</catalog-item-id><title>Vector Digital Data Template</title><catalog-item-type>Data Set</catalog-item-type><metadata-workflow-state ccs-id="4">Draft</metadata-workflow-state><parent-catalog-item-id/><parent-title>NOAA Distribution Mapping and Analysis Portal (DisMAP)</parent-title><parent-catalog-item-type>Project</parent-catalog-item-type><status>Completed</status><publication-date>2025-05-08</publication-date><abstract>Description of Vector Digital Data Template</abstract><purpose>Purpose Vector Digital Data Template</purpose></item-identification><keywords><keyword controlled="Yes"><keyword-type>Theme</keyword-type><thesaurus>ISO 19115 Topic Category</thesaurus><keyword>environment</keyword></keyword><

### Get path from root ###
#### /inport/api/get-path-from-root ####
#### (POST) ####

In [ ]:
import requests, json, copy
inport_url = "https://test-www.fisheries.noaa.gov/inport"
inport_response = requests.get(url=inport_url, auth=(credentials["username"], credentials["password"]))
if inport_response.status_code == 200:
    print("Authentication successful!")
    _url = "https://test-www.fisheries.noaa.gov/inport/api/get-session-id"
    session_id_response = requests.post(_url, json = credentials)
    session_id_dict = copy.deepcopy(session_id_response.json())
    print(session_id_dict)
    session_id_response.close()
    del _url, session_id_response    
    session_id_dict["catId"] = "75728"
    _url = "https://test-www.fisheries.noaa.gov/inport/api/get-path-from-root"
    response = requests.get(_url, params = session_id_dict)        
    #print(response.json())
    _dict = copy.deepcopy(response.json())
    
    response.close()
    del response, _url
    
    for key in _dict:
        print(key)
        if isinstance(_dict[key], dict):
            for k in _dict[key]:
                print(f"\t{k}: {_dict[key][k]}")
        elif isinstance(_dict[key], list):
            for i in range(0, len(_dict[key])):
                for k in _dict[key][i]:
                    print(f"\t{k}: {_dict[key][i][k]}")
                print()


elif inport_response.status_code == 401:
    print("Authentication NOT successful!")
else:
    print(f"Something is wrong!! Status Code: {response.status_code}")
inport_response.close()
del inport_response, inport_url
# RESULT 
# Authentication successful!
# {'sessionId': '75c3628752d91a126fcbded22998f3e'}
# path
# 	catId: 1196
# 	catItemType: LIB
# 	title: Office of Science and Technology Metadata Library
# 	shortName: OST Metadata Library
# 	ownerOrgAcronym: OST

## Contacts API Specs ###

### Endpoints ###

#### /inport/api/find-contact ####
#### (POST) ####

In [ ]:
import requests, json, copy
inport_url = "https://test-www.fisheries.noaa.gov/inport"
inport_response = requests.get(url=inport_url, auth=(credentials["username"], credentials["password"]))
if inport_response.status_code == 200:
    print("Authentication successful!")
    _url = "https://test-www.fisheries.noaa.gov/inport/api/get-session-id"
    session_id_response = requests.post(_url, json = credentials)
    session_id_dict = copy.deepcopy(session_id_response.json())
    print(session_id_dict)
    session_id_response.close()
    del _url, session_id_response    
    
    session_id_dict["contactType"]  = "Person"
    session_id_dict["emailAddress"] = "john.f.kennedy@noaa.gov"
    
    _url = "https://test-www.fisheries.noaa.gov/inport/api/find-contact"
    response = requests.get(_url, params = session_id_dict)        
    #print(response.json())
    _dict = copy.deepcopy(response.json())
    
    response.close()
    del response, _url
    
    for key in _dict:
        print(key)
        if isinstance(_dict[key], dict):
            for k in _dict[key]:
                print(f"\t{k}: {_dict[key][k]}")
        elif isinstance(_dict[key], list):
            for i in range(0, len(_dict[key])):
                for k in _dict[key][i]:
                    print(f"\t{k}: {_dict[key][i][k]}")


elif inport_response.status_code == 401:
    print("Authentication NOT successful!")
else:
    print(f"Something is wrong!! Status Code: {response.status_code}")
inport_response.close()
del inport_response, inport_url
# RESULT 
# Authentication successful!
# {'sessionId': '75c3628752d91a126fcbded22998f3e'}
# results
# 	lastName: Kennedy
# 	firstName: John
# 	middleName: F
# 	phone: 301-427-8149
# 	emailAddress: john.f.kennedy@noaa.gov
# 	address: 1315 East-West Highway
# 	city: Silver Spring
# 	stateProvince: MD
# 	postalCode: 20910
# 	country: USA
# 	contactType: Person
# 	rorMaxLength: 9
# 	orcIdMaxLength: 19

### Endpoints ###

#### /inport/api/find-person ####
#### (POST) ####

In [ ]:
import requests, json, copy
inport_url = "https://test-www.fisheries.noaa.gov/inport"
inport_response = requests.get(url=inport_url, auth=(credentials["username"], credentials["password"]))
if inport_response.status_code == 200:
    print("Authentication successful!")
    _url = "https://test-www.fisheries.noaa.gov/inport/api/get-session-id"
    session_id_response = requests.post(_url, json = credentials)
    session_id_dict = copy.deepcopy(session_id_response.json())
    #print(session_id_dict)
    session_id_response.close()
    del _url, session_id_response       
    
    session_id_dict["firstName"]  = "John"
    session_id_dict["middleName"] = "F"
    session_id_dict["lastName"]   = "Kennedy"
    print(session_id_dict)

    _url = "https://test-www.fisheries.noaa.gov/inport/api/find-person"
    response = requests.get(_url, params = session_id_dict)        
    print(response.json())
    _dict = copy.deepcopy(response.json())
    response.close()
    del response, _url
    
    for key in _dict:
        print(key)
        if isinstance(_dict[key], dict):
            for k in _dict[key]:
                print(f"\t{k}: {_dict[key][k]}")
        elif isinstance(_dict[key], list):
            for i in range(0, len(_dict[key])):
                for k in _dict[key][i]:
                    print(f"\t{k}: {_dict[key][i][k]}")


elif inport_response.status_code == 401:
    print("Authentication NOT successful!")
else:
    print(f"Something is wrong!! Status Code: {response.status_code}")
inport_response.close()
del inport_response, inport_url
# RESULT 
# Authentication successful!
# {'sessionId': '75c3628752d91a126fcbded22998f3e'}
# results
# 	lastName: Kennedy
# 	firstName: John
# 	middleName: F
# 	phone: 301-427-8149
# 	emailAddress: john.f.kennedy@noaa.gov
# 	address: 1315 East-West Highway
# 	city: Silver Spring
# 	stateProvince: MD
# 	postalCode: 20910
# 	country: USA
# 	contactType: Person
# 	rorMaxLength: 9
# 	orcIdMaxLength: 19

#### /inport/api/find-organization ####
#### (POST) ####

In [ ]:
import requests, json, copy
inport_url = "https://test-www.fisheries.noaa.gov/inport"
inport_response = requests.get(url=inport_url, auth=(credentials["username"], credentials["password"]))
if inport_response.status_code == 200:
    print("Authentication successful!")
    _url = "https://test-www.fisheries.noaa.gov/inport/api/get-session-id"
    session_id_response = requests.post(_url, json = credentials)
    session_id_dict = copy.deepcopy(session_id_response.json())
    #print(session_id_dict)
    session_id_response.close()
    del _url, session_id_response       
    
    session_id_dict["organizationName"]  = "NMFS Office of Science and Technology"
    print(session_id_dict)

    _url = "https://test-www.fisheries.noaa.gov/inport/api/find-organization"
    response = requests.get(_url, params = session_id_dict)        
    print(response.json())
    _dict = copy.deepcopy(response.json())
    response.close()
    del response, _url
    
    for key in _dict:
        print(key)
        if isinstance(_dict[key], dict):
            for k in _dict[key]:
                print(f"\t{k}: {_dict[key][k]}")
        elif isinstance(_dict[key], list):
            for i in range(0, len(_dict[key])):
                for k in _dict[key][i]:
                    print(f"\t{k}: {_dict[key][i][k]}")


elif inport_response.status_code == 401:
    print("Authentication NOT successful!")
else:
    print(f"Something is wrong!! Status Code: {response.status_code}")
inport_response.close()
del inport_response, inport_url
# RESULT 
# Authentication successful!
# {'sessionId': 'f0a9190579b46115943bad0f', 'organizationName': 'NMFS Office of Science and Technology'}
# {'results': [{'organizationName': 'NMFS Office of Science and Technology', 'noaaAcronym': 'OST', 'phone': '301-427-8100', 'address': '1315 East West Highway', 'city': 'Silver Spring', 'stateProvince': 'MD', 'postalCode': '20910', 'country': 'USA', 'url': 'https://www.st.nmfs.noaa.gov/', 'businessHours': '0700 - 1800 EST/EDT', 'contactType': 'Organization', 'rorMaxLength': 9, 'orcIdMaxLength': 19}]}
# results
# 	organizationName: NMFS Office of Science and Technology
# 	noaaAcronym: OST
# 	phone: 301-427-8100
# 	address: 1315 East West Highway
# 	city: Silver Spring
# 	stateProvince: MD
# 	postalCode: 20910
# 	country: USA
# 	url: https://www.st.nmfs.noaa.gov/
# 	businessHours: 0700 - 1800 EST/EDT
# 	contactType: Organization
# 	rorMaxLength: 9
# 	orcIdMaxLength: 19

#### /inport/api/find-organization-acronym ####
#### (POST) ####

In [ ]:
import requests, json, copy
inport_url = "https://test-www.fisheries.noaa.gov/inport"
inport_response = requests.get(url=inport_url, auth=(credentials["username"], credentials["password"]))
if inport_response.status_code == 200:
    print("Authentication successful!")
    _url = "https://test-www.fisheries.noaa.gov/inport/api/get-session-id"
    session_id_response = requests.post(_url, json = credentials)
    session_id_dict = copy.deepcopy(session_id_response.json())
    #print(session_id_dict)
    session_id_response.close()
    del _url, session_id_response       
    
    session_id_dict["noaaAcronym"] = "OST"
    print(session_id_dict)

    _url = "https://test-www.fisheries.noaa.gov/inport/api/find-organization-acronym"
    response = requests.get(_url, params = session_id_dict)        
    print(response.json())
    _dict = copy.deepcopy(response.json())
    response.close()
    del response, _url
    
    for key in _dict:
        print(key)
        if isinstance(_dict[key], dict):
            for k in _dict[key]:
                print(f"\t{k}: {_dict[key][k]}")
        elif isinstance(_dict[key], list):
            for i in range(0, len(_dict[key])):
                for k in _dict[key][i]:
                    print(f"\t{k}: {_dict[key][i][k]}")


elif inport_response.status_code == 401:
    print("Authentication NOT successful!")
else:
    print(f"Something is wrong!! Status Code: {response.status_code}")
inport_response.close()
del inport_response, inport_url
# RESULT 
# Authentication successful!
# {'sessionId': 'f0a9190579b46115943bad0f', 'organizationName': 'NMFS Office of Science and Technology'}
# {'results': [{'organizationName': 'NMFS Office of Science and Technology', 'noaaAcronym': 'OST', 'phone': '301-427-8100', 'address': '1315 East West Highway', 'city': 'Silver Spring', 'stateProvince': 'MD', 'postalCode': '20910', 'country': 'USA', 'url': 'https://www.st.nmfs.noaa.gov/', 'businessHours': '0700 - 1800 EST/EDT', 'contactType': 'Organization', 'rorMaxLength': 9, 'orcIdMaxLength': 19}]}
# results
# 	organizationName: NMFS Office of Science and Technology
# 	noaaAcronym: OST
# 	phone: 301-427-8100
# 	address: 1315 East West Highway
# 	city: Silver Spring
# 	stateProvince: MD
# 	postalCode: 20910
# 	country: USA
# 	url: https://www.st.nmfs.noaa.gov/
# 	businessHours: 0700 - 1800 EST/EDT
# 	contactType: Organization
# 	rorMaxLength: 9
# 	orcIdMaxLength: 19

#### /inport/api/find-position
#### (POST)
#### NOTE: not sure how to look for "position"

In [ ]:
import requests, json, copy
inport_url = "https://test-www.fisheries.noaa.gov/inport"
inport_response = requests.get(url=inport_url, auth=(credentials["username"], credentials["password"]))
if inport_response.status_code == 200:
    print("Authentication successful!")
    _url = "https://test-www.fisheries.noaa.gov/inport/api/get-session-id"
    session_id_response = requests.post(_url, json = credentials)
    session_id_dict = copy.deepcopy(session_id_response.json())
    #print(session_id_dict)
    session_id_response.close()
    del _url, session_id_response       
    
    session_id_dict["positionName"] = "Geologist"
    print(session_id_dict)

    _url = "https://test-www.fisheries.noaa.gov/inport/api/find-position"
    response = requests.get(_url, params = session_id_dict)        
    print(response.json())
    _dict = copy.deepcopy(response.json())
    response.close()
    del response, _url
    
    for key in _dict:
        print(key)
        if isinstance(_dict[key], dict):
            for k in _dict[key]:
                print(f"\t{k}: {_dict[key][k]}")
        elif isinstance(_dict[key], list):
            for i in range(0, len(_dict[key])):
                for k in _dict[key][i]:
                    print(f"\t{k}: {_dict[key][i][k]}")

elif inport_response.status_code == 401:
    print("Authentication NOT successful!")
else:
    print(f"Something is wrong!! Status Code: {response.status_code}")
inport_response.close()
del inport_response, inport_url
# RESULT 
# Authentication successful!
# {'sessionId': 'f0a9190579b46115943bad0f', 'organizationName': 'NMFS Office of Science and Technology'}
# {'results': [{'organizationName': 'NMFS Office of Science and Technology', 'noaaAcronym': 'OST', 'phone': '301-427-8100', 'address': '1315 East West Highway', 'city': 'Silver Spring', 'stateProvince': 'MD', 'postalCode': '20910', 'country': 'USA', 'url': 'https://www.st.nmfs.noaa.gov/', 'businessHours': '0700 - 1800 EST/EDT', 'contactType': 'Organization', 'rorMaxLength': 9, 'orcIdMaxLength': 19}]}
# results
# 	organizationName: NMFS Office of Science and Technology
# 	noaaAcronym: OST
# 	phone: 301-427-8100
# 	address: 1315 East West Highway
# 	city: Silver Spring
# 	stateProvince: MD
# 	postalCode: 20910
# 	country: USA
# 	url: https://www.st.nmfs.noaa.gov/
# 	businessHours: 0700 - 1800 EST/EDT
# 	contactType: Organization
# 	rorMaxLength: 9
# 	orcIdMaxLength: 19

#### /inport/api/add-contact ####
#### (POST) ####
##### Note: not used at this time #####

## Search API Specs ###

### Endpoints ###

#### /inport/api/search ####
#### (GET) ####

In [ ]:
import requests, json, copy
inport_url = "https://test-www.fisheries.noaa.gov/inport"
inport_response = requests.get(url=inport_url, auth=(credentials["username"], credentials["password"]))
if inport_response.status_code == 200:
    print("Authentication successful!")
    _url = "https://test-www.fisheries.noaa.gov/inport/api/get-session-id"
    session_id_response = requests.post(_url, json = credentials)
    session_id_dict = copy.deepcopy(session_id_response.json())
    #print(session_id_dict)
    session_id_response.close()
    del _url, session_id_response       
    
    session_id_dict["keywords"]    = "environment"
    session_id_dict["org"]         = "OST"
    session_id_dict["catItemType"] = "DS,ENT,PRJ"
    print(session_id_dict)

    _url = "https://test-www.fisheries.noaa.gov/inport/api/search"
    response = requests.get(_url, params = session_id_dict)        
    #print(response.json())
    _dict = copy.deepcopy(response.json())
    response.close()
    del response, _url
    
    for key in _dict:
        print(key)
        if isinstance(_dict[key], dict):
            for k in _dict[key]:
                print(f"\t{k}: {_dict[key][k]}")
        elif isinstance(_dict[key], list):
            for i in range(0, len(_dict[key])):
                for k in _dict[key][i]:
                    print(f"\t{k}: {_dict[key][i][k]}")

elif inport_response.status_code == 401:
    print("Authentication NOT successful!")
else:
    print(f"Something is wrong!! Status Code: {response.status_code}")
inport_response.close()
del inport_response, inport_url
# RESULT 
# Authentication successful!
# {'sessionId': '75c3628752d91a126fcbded22998f3e'}
# results
# 	catId: 20784
# 	catItemType: Project
# 	org: OST
# 	title: Large Marine Ecosystems (LME)
# 	abstractValue: Large Marine Ecosystems (LME) are ocean regions of 200,000 km2 or greater that are defined by ecological criteria, including bathymetry, hydrography, marine productivity, and trophically linked populations.  Since 1984, NOAA?s LME Program has developed ecosystem management tools, initiated projects that have been funded by partner organizations, and provided training for developing country part...